# Can you predict the likelihood of a customer defaulting on a loan based on their financial data ?

Entry of John Paul Curada for African Credit Scoring Challenge 

## I. Background

The African financial landscape is undergoing significant transformation with the rise of digital lending platforms and increased financial inclusion initiatives. However, assessing creditworthiness remains a complex challenge due to several unique factors:

Traditional credit scoring methods often fall short in the African context because:
1. Many potential borrowers lack conventional credit histories
2. Economic conditions vary dramatically across different regions
3. Market dynamics can shift rapidly, affecting borrowers' repayment capabilities
4. Limited access to formal banking systems creates gaps in financial data

A private asset manager operating across African financial markets has presented this challenge to develop more sophisticated approaches to credit risk assessment. The goal is to leverage machine learning to analyze both traditional loan data and macroeconomic indicators, creating a more comprehensive risk evaluation system.




## II. Objectives

The challenge has three primary objectives:

1. **Model Development**
   - Create a machine learning model that accurately predicts loan default probabilities
   - Develop a solution that can generalize across different African markets
   - Achieve high performance as measured by the F1 Score metric
   - Balance model complexity with interpretability

2. **Credit Scoring System**
   - Design a scalable credit scoring function for the top 10 winners
   - Transform model probabilities into actionable risk categories
   - Create a system that can adapt to different market conditions
   - Ensure the scoring system is transparent and implementable

3. **Risk Assessment Framework**
   - Incorporate both individual loan characteristics and economic indicators
   - Identify key factors that contribute to default risk
   - Create a framework that can support rapid decision-making
   - Enable expansion into new markets while managing risk effectively

Success in this challenge will contribute to more efficient lending practices across African markets, potentially increasing access to credit while maintaining sustainable risk levels for financial institutions.


## III. About Datasets

The analysis utilizes three primary datasets that provide comprehensive information about loan transactions, customer details, and economic indicators:

1. **Train.csv/Test.csv**  
These files contain the core loan and customer information, consisting of 16 variables. The dataset includes unique identifiers, loan details (amount, type, duration), customer information, and lender-specific data. The target variable indicates whether a customer defaulted (1) or paid (0) their loan. This data forms the foundation for building our predictive model.

| Variable Name | Description |
|------------|-------------|
| ID | A unique identifier for each entry in the dataset. |
| customer_id | Unique identifier for each customer in the dataset. |
| country_id | Identifier or code representing the country where the customer resides or where the loan was issued. |
| tbl_loan_id | Unique identifier for each loan associated with the customer. |
| Total_Amount | The total loan amount initially disbursed to the customer. |
| Total_Amount_to_Repay | The total amount the customer is expected to repay, including principal, interest, and fees. |
| loan_type | The category or type of loan. |
| disbursement_date | The date when the loan amount was disbursed to the customer. |
| duration | The length of the loan term, typically expressed in days |
| lender_id | Unique identifier for the lender or institution that issued the loan. |
| New_versus_Repeat | Indicates whether the loan is the customer's first loan ("New") or if the customer has taken loans before ("Repeat"). |
| Amount_Funded_By_Lender | The portion of the loan funded directly by the lender. |
| Lender_portion_Funded | Percentage of the total loan amount funded by the lender. |
| due_date | The date by which the loan repayment is due. |
| Lender_portion_to_be_repaid | The portion of the outstanding loan that needs to be repaid to the lender. |
| target | This variables takes the value 0 or 1. 1 means the customer defaulted on the loan, whereas 0 means, the customer paid the loan. |

2. **economic_indicators.csv**  
This dataset provides crucial macroeconomic context through nine key economic indicators for different countries. It includes:

| Variable Name | Description |
|------------|-------------|
| FP.CPI.TOTL.ZG | Inflation, consumer prices (annual %) |
| PA.NUS.FCRF | Official exchange rate (LCU per US$, period average) |
| FR.INR.RINR | Real interest rate (%) |
| AG.LND.PRCP.MM | Average precipitation in depth (mm per year) |
| FR.INR.DPST | Deposit interest rate (%) |
| FP.INR.LEND | Lending interest rate (%) |
| FR.INR.LNDP | Interest rate spread (lending rate minus deposit rate, %) |
| EG.USE.COMM.FO.ZS | Fossil fuel energy consumption (% of total) |
| SL.UEM.TOTL.ZS | Unemployment rate |

These indicators can significantly influence a borrower's ability to repay loans, as they reflect the overall economic health of the country where the loan was issued.

The combination of individual loan data and macroeconomic indicators allows for a more comprehensive analysis of loan default risk, taking into account both personal financial circumstances and broader economic conditions that might affect loan repayment behavior.



## Install Dependencies

In [1]:
#--------------------------------------
# Install required packages
#--------------------------------------
!pip install exploralytics

#--------------------------------------
# Custom libraries
#--------------------------------------
# My own made open-source project/library for plotly viz
from exploralytics import Visualizer 

#--------------------------------------
# Data manipulation and analysis
#--------------------------------------
import pandas as pd
import numpy as np

#--------------------------------------
# Visualization libraries
#--------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

#--------------------------------------
# Scikit-learn preprocessing
#--------------------------------------
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import (
   StandardScaler, 
   MinMaxScaler, 
   RobustScaler, 
   OneHotEncoder
)
from sklearn.impute import SimpleImputer, KNNImputer

#--------------------------------------
# Metrics and evaluation
#--------------------------------------
from sklearn.metrics import (
   accuracy_score, 
   precision_score, 
   recall_score, 
   f1_score,
   roc_curve, 
   precision_recall_curve, 
   auc, 
   roc_auc_score,
   confusion_matrix, 
   classification_report
)
from sklearn.model_selection import cross_val_score

#--------------------------------------
# Machine Learning Models
#--------------------------------------
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier 
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

#--------------------------------------
# Configuration and Settings
#--------------------------------------
import warnings
warnings.filterwarnings('ignore')

In [2]:
train_df = pd.read_csv('/kaggle/input/african-data-challenge/Train.csv')
test_df = pd.read_csv('/kaggle/input/african-data-challenge/Test.csv')
econ_in_df = pd.read_csv('/kaggle/input/african-data-challenge/economic_indicators.csv')

# Display the first few rows of the datasets and their shape
display(f"Train: {train_df.shape}", train_df.head(),
        f"Test: {test_df.shape}", test_df.head(),
        f"Economic Indicators: {econ_in_df.shape}", econ_in_df.head())

'Train: (68654, 16)'

,ID,customer_id,country_id,tbl_loan_id,lender_id,loan_type,Total_Amount,Total_Amount_to_Repay,disbursement_date,due_date,duration,New_versus_Repeat,Amount_Funded_By_Lender,Lender_portion_Funded,Lender_portion_to_be_repaid,target
0,ID_266671248032267278,266671,Kenya,248032,267278,Type_1,8448.0,8448.0,2022-08-30,2022-09-06,7,Repeat Loan,120.85,0.014305,121.0,0
1,ID_248919228515267278,248919,Kenya,228515,267278,Type_1,25895.0,25979.0,2022-07-30,2022-08-06,7,Repeat Loan,7768.50,0.300000,7794.0,0
2,ID_308486370501251804,308486,Kenya,370501,251804,Type_7,6900.0,7142.0,2024-09-06,2024-09-13,7,Repeat Loan,1380.00,0.200000,1428.0,0
3,ID_266004285009267278,266004,Kenya,285009,267278,Type_1,8958.0,9233.0,2022-10-20,2022-10-27,7,Repeat Loan,2687.40,0.300000,2770.0,0
4,ID_253803305312267278,253803,Kenya,305312,267278,Type_1,4564.0,4728.0,2022-11-28,2022-12-05,7,Repeat Loan,1369.20,0.300000,1418.0,0


'Test: (18594, 15)'

,ID,customer_id,country_id,tbl_loan_id,lender_id,loan_type,Total_Amount,Total_Amount_to_Repay,disbursement_date,due_date,duration,New_versus_Repeat,Amount_Funded_By_Lender,Lender_portion_Funded,Lender_portion_to_be_repaid
0,ID_269404226088267278,269404,Kenya,226088,267278,Type_1,1919.0,1989.0,2022-07-27,2022-08-03,7,Repeat Loan,575.7,0.300000,597.0
1,ID_255356300042267278,255356,Kenya,300042,267278,Type_1,2138.0,2153.0,2022-11-16,2022-11-23,7,Repeat Loan,0.0,0.000000,0.0
2,ID_257026243764267278,257026,Kenya,243764,267278,Type_1,8254.0,8304.0,2022-08-24,2022-08-31,7,Repeat Loan,207.0,0.025079,208.0
3,ID_264617299409267278,264617,Kenya,299409,267278,Type_1,3379.0,3379.0,2022-11-15,2022-11-22,7,Repeat Loan,1013.7,0.300000,1014.0
4,ID_247613296713267278,247613,Kenya,296713,267278,Type_1,120.0,120.0,2022-11-10,2022-11-17,7,Repeat Loan,36.0,0.300000,36.0


'Economic Indicators: (27, 25)'

,Country,Indicator,YR2001,YR2002,YR2003,YR2004,YR2005,YR2006,YR2007,YR2008,...,YR2014,YR2015,YR2016,YR2017,YR2018,YR2019,YR2020,YR2021,YR2022,YR2023
0,Ghana,"Inflation, consumer prices (annual %)",41.509496,9.360932,29.772980,18.042739,15.438992,11.679184,10.734267,16.494640,...,15.489616,17.149970,17.454635,12.371922,7.808765,7.143640,9.887290,9.971089,31.255895,38.106966
1,Cote d'Ivoire,"Inflation, consumer prices (annual %)",4.361529,3.077265,3.296807,1.457988,3.885830,2.467191,1.892006,6.308528,...,0.448682,1.251500,0.723178,0.685881,0.359409,-1.106863,2.425007,4.091952,5.276167,4.387117
2,Kenya,"Inflation, consumer prices (annual %)",5.738598,1.961308,9.815691,11.624036,10.312778,14.453734,9.758880,26.239817,...,6.878155,6.582154,6.297250,8.005650,4.689806,5.239638,5.405162,6.107936,7.659863,7.671396
3,Ghana,"Official exchange rate (LCU per US$, period av...",0.716305,0.792417,0.866764,0.899495,0.905209,0.915107,0.932619,1.052275,...,2.896575,3.714642,3.909817,4.350533,4.585325,5.217367,5.595708,5.805700,8.272400,11.020408
4,Cote d'Ivoire,"Official exchange rate (LCU per US$, period av...",732.397693,693.713226,579.897426,527.338032,527.258363,522.425625,478.633718,446.000041,...,493.757330,591.211698,592.605615,580.656750,555.446458,585.911013,575.586005,554.530675,623.759701,606.569750


## IV.Exploratory Data Analysis

### 1. Data Quality Checks



#### a. Missing Values

In [21]:
def print_missing_summary(df, df_name=None):
   """
   Prints missing values summary with counts and percentages.
   """
   total_rows = len(df)
   # Use df_name if provided, otherwise try to get the variable name
   name = df_name or getattr(df, '__name__', 'DataFrame')
   print(f"{name}")
   print(f"Total rows: {total_rows}")
   print("\nMissing Values Summary: ")
   print("-" * 50)
   max_col_length = max(len(str(col)) for col in df.columns)
   for col in df.columns:
       missing = df[col].isnull().sum()
       percent = (missing/total_rows * 100).round(2)
       print(f"{str(col):<{max_col_length}}\t\t{missing} missing ({percent}%)")


In [22]:
print_missing_summary(train_df, "Training data (train_df)")
print_missing_summary(test_df, "Testing data (test_df)")
print_missing_summary(econ_in_df, "Economic Indicator (econ_in_df")

Training data (train_df)
Total rows: 68654

Missing Values Summary: 
--------------------------------------------------
ID                         		0 missing (0.0%)
customer_id                		0 missing (0.0%)
country_id                 		0 missing (0.0%)
tbl_loan_id                		0 missing (0.0%)
lender_id                  		0 missing (0.0%)
loan_type                  		0 missing (0.0%)
Total_Amount               		0 missing (0.0%)
Total_Amount_to_Repay      		0 missing (0.0%)
disbursement_date          		0 missing (0.0%)
due_date                   		0 missing (0.0%)
duration                   		0 missing (0.0%)
New_versus_Repeat          		0 missing (0.0%)
Amount_Funded_By_Lender    		0 missing (0.0%)
Lender_portion_Funded      		0 missing (0.0%)
Lender_portion_to_be_repaid		0 missing (0.0%)
target                     		0 missing (0.0%)
Testing data (test_df)
Total rows: 18594

Missing Values Summary: 
--------------------------------------------------
ID                        

### b. Data Types

In [23]:
def convert_datatypes(df):
   """
   Convert DataFrame columns to appropriate data types
   """
   # Create a copy to avoid modifying original
   df = df.copy()

   # Convert ID columns to string/object
   id_columns = ['ID', 'country_id', 'customer_id','tbl_loan_id']
   df[id_columns] = df[id_columns].astype(str)

   # Convert numeric ID columns to int64
   numeric_id_columns = ['duration']  # Removed customer_id
   df[numeric_id_columns] = df[numeric_id_columns].astype('int64')

   # Convert float columns
   float_columns = [
       'Total_Amount',
       'Total_Amount_to_Repay',
       'Amount_Funded_By_Lender',
       'Lender_portion_Funded',
       'Lender_portion_to_be_repaid'
   ]
   df[float_columns] = df[float_columns].astype('float64')

   # Convert date columns to datetime
   date_columns = ['disbursement_date', 'due_date']
   df[date_columns] = df[date_columns].apply(pd.to_datetime)

   # Convert categorical columns
   df['loan_type'] = df['loan_type'].astype('category')
   df['New_versus_Repeat'] = df['New_versus_Repeat'].astype('category')
   df['lender_id'] = df['lender_id'].astype('category')

   return df

In [24]:
# Apply the conversion
train_df = convert_datatypes(train_df)

# Verify the changes
print("Dataset Info After Type Conversion:")
print(train_df.info())

# Memory usage before and after
print("\nMemory Usage:")
print(f"Memory usage: {train_df.memory_usage().sum() / 1024**2:.2f} MB")

Dataset Info After Type Conversion:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68654 entries, 0 to 68653
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   ID                           68654 non-null  object        
 1   customer_id                  68654 non-null  object        
 2   country_id                   68654 non-null  object        
 3   tbl_loan_id                  68654 non-null  object        
 4   lender_id                    68654 non-null  category      
 5   loan_type                    68654 non-null  category      
 6   Total_Amount                 68654 non-null  float64       
 7   Total_Amount_to_Repay        68654 non-null  float64       
 8   disbursement_date            68654 non-null  datetime64[ns]
 9   due_date                     68654 non-null  datetime64[ns]
 10  duration                     68654 non-null  int64         
 11  New_v

### c. Basic Statistics

In [25]:
def print_readable_stats(df):
    """
    Print formatted descriptive statistics including median
    """
    # Select only numeric columns
    numeric_df = df.select_dtypes(include=['int64', 'float64', 'int32', 'float32'])

    stats = numeric_df.describe()

    # Format numbers with comma separators
    formatted_stats = pd.DataFrame({
        'Count': stats.loc['count'].map('{:,.0f}'.format),
        'Mean': stats.loc['mean'].map('{:,.2f}'.format),
        'Median': numeric_df.median().map('{:,.2f}'.format),
        'Std': stats.loc['std'].map('{:,.2f}'.format),
        'Min': stats.loc['min'].map('{:,.2f}'.format),
        '25%': stats.loc['25%'].map('{:,.2f}'.format),
        '75%': stats.loc['75%'].map('{:,.2f}'.format),
        'Max': stats.loc['max'].map('{:,.2f}'.format)
    })

    # Print with clean formatting
    print("\nDescriptive Statistics for Numeric Columns:")
    print("----------------------------------------")
    print(formatted_stats.to_string())

    # Additional info per column
    print("\nDetailed Column Information:")
    print("--------------------------")
    for col in numeric_df.columns:
        print(f"\n{col}:")
        print(f"Skewness: {numeric_df[col].skew():.2f}")
        print(f"Missing Values: {numeric_df[col].isnull().sum():,} ({numeric_df[col].isnull().mean()*100:.1f}%)")


# Usage
print_readable_stats(train_df)


Descriptive Statistics for Numeric Columns:
----------------------------------------
                              Count       Mean    Median         Std   Min       25%        75%            Max
Total_Amount                 68,654  14,836.83  5,249.00  141,649.87  2.00  2,295.00  11,450.00  23,000,000.00
Total_Amount_to_Repay        68,654  15,639.93  5,325.00  165,078.35  0.00  2,329.00  11,650.00  25,415,000.00
duration                     68,654       8.54      7.00       13.34  1.00      7.00       7.00       1,096.00
Amount_Funded_By_Lender      68,654   2,545.66    915.00   11,922.72  0.00    234.00   2,272.65   1,600,000.00
Lender_portion_Funded        68,654       0.22      0.30        0.13  0.00      0.12       0.30           1.17
Lender_portion_to_be_repaid  68,654   2,652.62    934.00   13,380.06  0.00    239.00   2,317.00   1,821,338.00
target                       68,654       0.02      0.00        0.13  0.00      0.00       0.00           1.00

Detailed Column Informati

we might NOT need to remove outliers because
- Extreme values in loan amounts could be legitimate high-value loans
- Longer durations might indicate different loan products
- Outliers might actually be predictive of default risk
- Removing them could hide important risk patterns

In [26]:
def plot_target_distribution(df, target):
   """
   Plots target class distribution with counts and percentages.
   Colors bars based on value - green for high, red for low.
   """
   # Get value counts and percentages
   counts = df[target].value_counts()
   percentages = df[target].value_counts(normalize=True) * 100

   # Create figure with count and percentage bars
   fig = go.Figure()

   # Set colors based on count values
   colors = ['#03EF62' if count == counts.max() else '#FF999C'
            for count in [counts[0], counts[1]]]

   # Add count bars
   fig.add_trace(go.Bar(
       x=[0, 1],  # Use actual values instead of unique method
       y=[counts[0], counts[1]],
       text=[f'n={counts[0]}<br>{percentages[0]:.1f}%',
             f'n={counts[1]}<br>{percentages[1]:.1f}%'],
       textposition='auto',
       name='Count',
       marker_color=colors
   ))

   # Update layout
   fig.update_layout(
       title='Target Distribution (0: Paid, 1: Default)',
       xaxis_title='Status',
       yaxis_title='Count',
       showlegend=False
   )

   fig.show()

   # Print summary
   print(f"\nClass Distribution Summary:")
   print("-" * 30)
   print(f"Paid (0): {counts[0]} ({percentages[0]:.1f}%)")
   print(f"Default (1): {counts[1]} ({percentages[1]:.1f}%)")

# Example usage
plot_target_distribution(train_df, 'target')


Class Distribution Summary:
------------------------------
Paid (0): 67396 (98.2%)
Default (1): 1258 (1.8%)


### Basic Exploration

In [29]:
def explore_datasets(train_df, test_df, econ_df):
    # 1. Customer Analysis
    train_customers = train_df['customer_id'].nunique()
    test_customers = test_df['customer_id'].nunique()
    common_customers = len(set(train_df['customer_id']).intersection(set(test_df['customer_id'])))
    
    print("Customer Analysis:")
    print(f"Train customers: {train_customers}")
    print(f"Test customers: {test_customers}")
    print(f"Common customers: {common_customers}")
    
    # 2. Basic Dataset Properties
    print("\nDataset Sizes:")
    print(f"Train shape: {train_df.shape}")
    print(f"Test shape: {test_df.shape}")
    print(f"Economic indicators shape: {econ_df.shape}")
    
    # 3. Loan Type Distribution
    print("\nLoan Type Distribution (Train):")
    print(train_df['loan_type'].value_counts(normalize=True).round(3))
    
    # 4. New vs Repeat Borrowers
    print("\nNew vs Repeat Borrowers (Train):")
    print(train_df['New_versus_Repeat'].value_counts(normalize=True).round(3))
    
    # 5. Default Rate Analysis
    print("\nDefault Rate Analysis (Train):")
    default_rate = train_df['target'].mean()
    print(f"Overall default rate: {default_rate:.2%}")
    
    # 6. Country Analysis
    print("\nCountry Distribution (Train):")
    print(train_df['country_id'].value_counts(normalize=True).round(3))
    
    # 7. Basic Loan Amount Statistics
    print("\nLoan Amount Statistics (Train):")
    amount_stats = train_df['Total_Amount'].describe()
    print(amount_stats)
    
    # 8. Duration Analysis
    print("\nLoan Duration Statistics (Train):")
    duration_stats = train_df['duration'].describe()
    print(duration_stats)
    
    # 9. Economic Indicators Coverage
    print("\nEconomic Indicators Coverage:")
    missing_econ = econ_df.isnull().sum() / len(econ_df)
    print(missing_econ.round(3))

    return {
        'train_customers': train_customers,
        'test_customers': test_customers,
        'common_customers': common_customers,
        'default_rate': default_rate
    }

# Usage:
stats = explore_datasets(train_df, test_df, econ_in_df)

Customer Analysis:
Train customers: 6540
Test customers: 4962
Common customers: 0

Dataset Sizes:
Train shape: (68654, 16)
Test shape: (18594, 15)
Economic indicators shape: (27, 25)

Loan Type Distribution (Train):
loan_type
Type_1     0.899
Type_7     0.041
Type_5     0.022
Type_4     0.018
Type_10    0.007
Type_6     0.005
Type_9     0.003
Type_14    0.001
Type_2     0.001
Type_11    0.001
Type_18    0.000
Type_17    0.000
Type_12    0.000
Type_23    0.000
Type_20    0.000
Type_16    0.000
Type_13    0.000
Type_19    0.000
Type_15    0.000
Type_21    0.000
Type_24    0.000
Type_22    0.000
Name: proportion, dtype: float64

New vs Repeat Borrowers (Train):
New_versus_Repeat
Repeat Loan    0.992
New Loan       0.008
Name: proportion, dtype: float64

Default Rate Analysis (Train):
Overall default rate: 1.83%

Country Distribution (Train):
country_id
Kenya    1.0
Name: proportion, dtype: float64

Loan Amount Statistics (Train):
count    6.865400e+04
mean     1.483683e+04
std      1.4164

In [32]:
def analyze_ids_and_loans(train_df, test_df):
    """
    Analyze IDs and loan types in the datasets
    """
    print("ID Analysis:")
    print("============")
    
    # Customer ID Analysis
    train_customers = train_df['customer_id'].nunique()
    test_customers = test_df['customer_id'].nunique()
    common_customers = len(set(train_df['customer_id']).intersection(set(test_df['customer_id'])))
    
    print(f"\nCustomer IDs:")
    print(f"Train unique customers: {train_customers:,}")
    print(f"Test unique customers: {test_customers:,}")
    print(f"Common customers: {common_customers:,}")
    print(f"Customer overlap: {(common_customers/train_customers*100):.2f}%")
    
    # Loan ID Analysis
    print(f"\nLoan IDs (tbl_loan_id):")
    print(f"Train unique loans: {train_df['tbl_loan_id'].nunique():,}")
    print(f"Test unique loans: {test_df['tbl_loan_id'].nunique():,}")
    common_loans = len(set(train_df['tbl_loan_id']).intersection(set(test_df['tbl_loan_id'])))
    print(f"Common loans: {common_loans:,}")
    
    # Lender ID Analysis
    print(f"\nLender IDs:")
    print(f"Train unique lenders: {train_df['lender_id'].nunique():,}")
    print(f"Test unique lenders: {test_df['lender_id'].nunique():,}")
    common_lenders = len(set(train_df['lender_id']).intersection(set(test_df['lender_id'])))
    print(f"Common lenders: {common_lenders:,}")
    
    print("\nLoan Type Analysis:")
    print("==================")
    
    # Loan Type Distribution
    print("\nLoan Type Distribution in Train:")
    loan_counts = train_df['loan_type'].value_counts()
    loan_percentages = train_df['loan_type'].value_counts(normalize=True) * 100
    
    for loan_type, count in loan_counts.items():
        print(f"{loan_type}: {count:,} ({loan_percentages[loan_type]:.2f}%)")
    
    # Default Rates by Loan Type
    print("\nDefault Rates by Loan Type:")
    default_rates = train_df.groupby('loan_type')['target'].mean() * 100
    for loan_type, rate in default_rates.items():
        print(f"{loan_type}: {rate:.2f}%")
        
    # Check for loan types present in test but not in train
    test_only_types = set(test_df['loan_type']) - set(train_df['loan_type'])
    if test_only_types:
        print("\nWARNING: Loan types in test but not in train:")
        print(test_only_types)

    return {
        'train_customers': train_customers,
        'test_customers': test_customers,
        'common_customers': common_customers,
        'loan_types': loan_counts.to_dict(),
        'default_rates': default_rates.to_dict()
    }

# Usage:
stats = analyze_ids_and_loans(train_df, test_df)

ID Analysis:

Customer IDs:
Train unique customers: 6,540
Test unique customers: 4,962
Common customers: 0
Customer overlap: 0.00%

Loan IDs (tbl_loan_id):
Train unique loans: 66,520
Test unique loans: 17,067
Common loans: 0

Lender IDs:
Train unique lenders: 4
Test unique lenders: 8
Common lenders: 4

Loan Type Analysis:

Loan Type Distribution in Train:
Type_1: 61,723 (89.90%)
Type_7: 2,790 (4.06%)
Type_5: 1,521 (2.22%)
Type_4: 1,235 (1.80%)
Type_10: 466 (0.68%)
Type_6: 357 (0.52%)
Type_9: 205 (0.30%)
Type_14: 99 (0.14%)
Type_2: 74 (0.11%)
Type_11: 42 (0.06%)
Type_18: 30 (0.04%)
Type_17: 23 (0.03%)
Type_12: 21 (0.03%)
Type_23: 18 (0.03%)
Type_20: 13 (0.02%)
Type_16: 12 (0.02%)
Type_13: 10 (0.01%)
Type_19: 4 (0.01%)
Type_15: 4 (0.01%)
Type_21: 3 (0.00%)
Type_24: 3 (0.00%)
Type_22: 1 (0.00%)

Default Rates by Loan Type:
Type_1: 0.84%
Type_10: 2.58%
Type_11: 4.76%
Type_12: 4.76%
Type_13: 0.00%
Type_14: 71.72%
Type_15: 75.00%
Type_16: 0.00%
Type_17: 0.00%
Type_18: 3.33%
Type_19: 0.00%
Ty

In [33]:
# Check percentage of test data affected
affected_pct = (test_df['loan_type'].isin(['Type_3', 'Type_8']).mean() * 100)
print(f"Percentage of test data affected: {affected_pct:.2f}%")

# Check loan_type importance in training
from sklearn.metrics import mutual_info_score
mi_score = mutual_info_score(train_df['loan_type'], train_df['target'])
print(f"Mutual information with target: {mi_score:.4f}")

Percentage of test data affected: 16.54%
Mutual information with target: 0.0170


### d.

## V. Data Preprocessing

In [ ]:
train_df.info()

In [ ]:
def convert_datatypes(df):
   """
   Convert DataFrame columns to appropriate data types
   """
   # Create a copy to avoid modifying original
   df = df.copy()

   # Convert ID columns to string/object
   id_columns = ['ID', 'country_id', 'customer_id', 'lender_id']
   df[id_columns] = df[id_columns].astype(str)

   # Convert numeric ID columns to int64
   numeric_id_columns = ['tbl_loan_id']  # Removed customer_id
   df[numeric_id_columns] = df[numeric_id_columns].astype('int64')

   # Convert float columns
   float_columns = [
       'Total_Amount',
       'Total_Amount_to_Repay',
       'Amount_Funded_By_Lender',
       'Lender_portion_Funded',
       'Lender_portion_to_be_repaid'
   ]
   df[float_columns] = df[float_columns].astype('float64')

   # Convert date columns to datetime
   date_columns = ['disbursement_date', 'due_date']
   df[date_columns] = df[date_columns].apply(pd.to_datetime)

   # Convert categorical columns
   df['loan_type'] = df['loan_type'].astype('category')
   df['New_versus_Repeat'] = df['New_versus_Repeat'].astype('category')

   # Convert target and duration to int
   df['target'] = df['target'].astype('int8')  # Using int8 since it's binary
   df['duration'] = df['duration'].astype('int32')

   return df

In [ ]:
train_df = convert_datatypes(train_df)

In [ ]:
sorted_train_df = train_df.sort_values(by='disbursement_date')
sorted_train_df['interest_and_fees'] = sorted_train_df['Total_Amount_to_Repay'] - sorted_train_df['Total_Amount']
sorted_train_df['interest_rate'] = (interest_and_fees / Total_Amount) * 100

In [ ]:
sorted_train_df['Total_Amount']

In [ ]:
train_df.query("customer_id=='136048'").sort_values(by='disbursement_date')

#

## playground

In [2]:
def create_aggregated_features(train_df, test_df=None):
    """
    Creates essential aggregated features for credit scoring model.
    """
    # Customer Behavior Features (most predictive for credit scoring)
    customer_agg = train_df.groupby('customer_id').agg({
        'tbl_loan_id': 'count',  # Number of loans - key indicator
        'Total_Amount': ['mean', 'max'],  # Loan amount behavior
        'duration': 'mean',  # Average loan duration
        'New_versus_Repeat': lambda x: (x == 'Repeat').mean()  # Repeat customer ratio
    }).round(3)
    
    # Flatten column names
    customer_agg.columns = [
        'loan_count',
        'avg_loan_amount', 
        'max_loan_amount',
        'avg_duration',
        'repeat_loan_ratio'
    ]
    
    # Essential Financial Ratios
    def calculate_financial_ratios(df):
        df = df.copy()
        df['repayment_to_loan_ratio'] = df['Total_Amount_to_Repay'] / df['Total_Amount']
        df['daily_repayment_amount'] = df['Total_Amount_to_Repay'] / df['duration']
        return df
    
    # Apply transformations
    if test_df is not None:
        df_to_transform = test_df.copy()
    else:
        df_to_transform = train_df.copy()
    
    # Calculate ratios and merge aggregations
    df_transformed = calculate_financial_ratios(df_to_transform)
    df_transformed = df_transformed.merge(customer_agg, on='customer_id', how='left')
    
    # Handle missing values
    df_transformed = df_transformed.fillna(0)
    
    # Core features for modeling
    features_for_modeling = [
        # Original features
        'Total_Amount', 
        'Total_Amount_to_Repay', 
        'duration',
        'Amount_Funded_By_Lender',
        
        # Key ratios
        'repayment_to_loan_ratio',
        'daily_repayment_amount',
        
        # Customer aggregations
        'loan_count',
        'avg_loan_amount',
        'max_loan_amount',
        'avg_duration',
        'repeat_loan_ratio'
    ]
    
    return df_transformed[features_for_modeling]

In [3]:
def create_aggregated_features(train_df, test_df=None):
    """
    Creates enhanced features for credit scoring with robust handling of numerical issues
    """
    def safe_divide(a, b):
        """Safe division handling zeros and infinities"""
        result = np.divide(a, b, out=np.zeros_like(a), where=b!=0)
        return np.clip(result, -1e15, 1e15)  # Clip extreme values
    
    # Customer Behavior and Risk Features
    customer_agg = train_df.groupby('customer_id').agg({
        'tbl_loan_id': 'count',
        'Total_Amount': ['mean', 'max', 'sum', 'std'],
        'duration': ['mean', 'max', 'std'],
        'New_versus_Repeat': lambda x: (x == 'Repeat').mean(),
        'Total_Amount_to_Repay': ['mean', 'sum'],
        'Amount_Funded_By_Lender': ['mean', 'sum']
    }).round(3)
    
    # Flatten column names
    customer_agg.columns = [
        'loan_count',
        'avg_loan_amount', 'max_loan_amount', 'total_loan_amount', 'std_loan_amount',
        'avg_duration', 'max_duration', 'std_duration',
        'repeat_loan_ratio',
        'avg_repayment_amount', 'total_repayment_burden',
        'avg_funded_amount', 'total_funded_amount'
    ]
    
    # Advanced Financial Ratios with safe division
    def calculate_financial_ratios(df):
        df = df.copy()
        
        # Basic ratios with safe division
        df['repayment_to_loan_ratio'] = safe_divide(df['Total_Amount_to_Repay'], df['Total_Amount'])
        df['daily_repayment_amount'] = safe_divide(df['Total_Amount_to_Repay'], df['duration'])
        df['funded_ratio'] = safe_divide(df['Amount_Funded_By_Lender'], df['Total_Amount'])
        
        # Risk indicators
        df['repayment_burden'] = safe_divide(df['Total_Amount_to_Repay'], df['duration'])
        df['funding_gap'] = df['Total_Amount'] - df['Amount_Funded_By_Lender']
        
        # Interest calculations
        df['interest_burden'] = df['Total_Amount_to_Repay'] - df['Total_Amount']
        df['interest_rate'] = safe_divide(df['interest_burden'], df['Total_Amount'])
        
        return df
    
    # Lender Risk Metrics
    lender_agg = train_df.groupby('lender_id').agg({
        'Total_Amount': ['mean', 'std'],
        'tbl_loan_id': 'count'
    }).round(3)
    
    lender_agg.columns = [
        'lender_avg_loan', 'lender_std_loan', 'lender_loan_count'
    ]
    
    # Apply transformations
    if test_df is not None:
        df_to_transform = test_df.copy()
    else:
        df_to_transform = train_df.copy()
    
    # Calculate and merge all features
    df_transformed = calculate_financial_ratios(df_to_transform)
    df_transformed = df_transformed.merge(customer_agg, on='customer_id', how='left')
    df_transformed = df_transformed.merge(lender_agg, on='lender_id', how='left')
    
    # Handle missing values and infinities
    numeric_cols = df_transformed.select_dtypes(include=['float64', 'int64']).columns
    df_transformed[numeric_cols] = df_transformed[numeric_cols].replace([np.inf, -np.inf], np.nan)
    
    # Fill missing values with median instead of mean (more robust to outliers)
    medians = df_transformed[numeric_cols].median()
    df_transformed[numeric_cols] = df_transformed[numeric_cols].fillna(medians)
    
    # Core features for modeling
    features_for_modeling = [
        # Original features
        'Total_Amount', 'Total_Amount_to_Repay', 'duration',
        'Amount_Funded_By_Lender',
        
        # Financial ratios
        'repayment_to_loan_ratio', 'daily_repayment_amount',
        'funded_ratio', 'repayment_burden', 'funding_gap',
        'interest_burden', 'interest_rate',
        
        # Customer history
        'loan_count', 'avg_loan_amount', 'max_loan_amount',
        'total_loan_amount', 'std_loan_amount', 'avg_duration',
        'max_duration', 'std_duration', 'repeat_loan_ratio',
        'avg_repayment_amount', 'total_repayment_burden',
        'avg_funded_amount', 'total_funded_amount',
        
        # Lender metrics
        'lender_avg_loan', 'lender_std_loan', 'lender_loan_count'
    ]
    
    # Final check for any remaining infinities
    df_final = df_transformed[features_for_modeling]
    df_final = df_final.clip(-1e15, 1e15)  # Clip any extreme values
    
    return df_final

In [4]:
def prepare_economic_data(econ_df):
    """
    Prepare economic data by melting year columns
    """
    # List of indicators
    INDICATORS = [
        'Inflation, consumer prices (annual %)',
        'Official exchange rate (LCU per US$, period average)',
        'Real interest rate (%)',
        'Average precipitation in depth (mm per year)',
        'Deposit interest rate (%)',
        'Lending interest rate (%)',
        'Interest rate spread (lending rate minus deposit rate, %)',
        'Fossil fuel energy consumption (% of total)',
        'Unemployment rate'
    ]
    
    # Keep original index and indicator names
    econ_df = econ_df.copy()
    
    # Identify year columns
    year_cols = [col for col in econ_df.columns if col.startswith('YR')]
    
    # Melt the dataframe
    melted_df = pd.melt(
        econ_df,
        id_vars=['Country', 'Indicator'],  # Changed from CountryIndicator
        value_vars=year_cols,
        var_name='year',
        value_name='value'
    )
    
    # Clean up year column
    melted_df['year'] = melted_df['year'].str.replace('YR', '').astype(int)
    
    # Create final economic indicators dataframe
    final_df = pd.pivot_table(
        melted_df,
        index=['Country', 'year'],  # Changed from CountryIndicator
        columns='Indicator',
        values='value'
    ).reset_index()
    
    # Rename columns for easier handling
    column_mapping = {
        'Country': 'country_id',  # Map Country to country_id
        'Inflation, consumer prices (annual %)': 'inflation_rate',
        'Official exchange rate (LCU per US$, period average)': 'exchange_rate',
        'Real interest rate (%)': 'real_interest_rate',
        'Average precipitation in depth (mm per year)': 'precipitation',
        'Deposit interest rate (%)': 'deposit_rate',
        'Lending interest rate (%)': 'lending_rate',
        'Interest rate spread (lending rate minus deposit rate, %)': 'interest_spread',
        'Fossil fuel energy consumption (% of total)': 'fossil_fuel_consumption',
        'Unemployment rate': 'unemployment_rate'
    }
    
    final_df = final_df.rename(columns=column_mapping)
    
    # Handle missing values
    numerical_cols = list(column_mapping.values())[1:]  # Exclude country_id
    final_df[numerical_cols] = final_df[numerical_cols].fillna(final_df[numerical_cols].mean())
    
    return final_df

# Let's check unique countries in both datasets
print("Economic Data Countries:", econ_in_df['Country'].unique())
print("Train Data Countries:", train_df['country_id'].unique())

NameError: name 'econ_in_df' is not defined

In [ ]:
def create_aggregated_features(train_df, test_df=None, econ_df=None):
    """
    Creates features combining loan data and economic indicators
    """
    def safe_divide(a, b):
        """Safe division handling zeros"""
        return np.divide(a, b, out=np.zeros_like(a), where=b!=0)
    
    # Customer Behavior Features
    customer_agg = train_df.groupby('customer_id').agg({
        'tbl_loan_id': ['count', 'nunique'],
        'Total_Amount': ['mean', 'sum'],
        'duration': ['mean', 'max', 'std'],
        'Total_Amount_to_Repay': 'mean',
        'New_versus_Repeat': lambda x: (x == 'Repeat').mean()
    }).round(3)
    
    # Flatten column names
    customer_agg.columns = [
        'loan_count', 'unique_loans',
        'avg_loan_amount', 'total_borrowed',
        'avg_duration', 'max_duration', 'duration_std',
        'avg_repayment_amount',
        'repeat_ratio'
    ]
    
    # Financial Ratios
    def calculate_financial_ratios(df):
        df = df.copy()
        
        # Core ratios (highest importance from previous analysis)
        df['repayment_to_loan_ratio'] = safe_divide(df['Total_Amount_to_Repay'], df['Total_Amount'])
        df['daily_repayment'] = safe_divide(df['Total_Amount_to_Repay'], df['duration'])
        df['daily_principal'] = safe_divide(df['Total_Amount'], df['duration'])
        
        # Interest metrics
        df['interest_amount'] = df['Total_Amount_to_Repay'] - df['Total_Amount']
        df['interest_rate'] = safe_divide(df['interest_amount'], df['Total_Amount'])
        df['daily_interest'] = safe_divide(df['interest_amount'], df['duration'])
        
        # Funding metrics
        df['funding_ratio'] = safe_divide(df['Amount_Funded_By_Lender'], df['Total_Amount'])
        df['funding_gap_ratio'] = 1 - df['funding_ratio']
        
        return df
    
    # Process data
    if test_df is not None:
        df_to_transform = test_df.copy()
    else:
        df_to_transform = train_df.copy()
    
    # Calculate features and merge aggregations
    df_transformed = calculate_financial_ratios(df_to_transform)
    df_transformed = df_transformed.merge(customer_agg, on='customer_id', how='left')
    
    # Add economic indicators
    if econ_df is not None:
        # Extract year from disbursement date
        df_transformed['year'] = pd.to_datetime(df_transformed['disbursement_date']).dt.year
        
        # Merge with economic data
        df_transformed = df_transformed.merge(
            econ_df,
            on=['country_id', 'year'],
            how='left'
        )
        
        # Create relative metrics using economic indicators
        df_transformed['interest_to_market_ratio'] = safe_divide(df_transformed['interest_rate'], 
                                                               df_transformed['lending_rate'])
        df_transformed['relative_funding_cost'] = df_transformed['interest_rate'] - df_transformed['lending_rate']
    
    # Handle missing values
    numeric_cols = df_transformed.select_dtypes(include=['float64', 'int64']).columns
    df_transformed[numeric_cols] = df_transformed[numeric_cols].replace([np.inf, -np.inf], np.nan)
    medians = df_transformed[numeric_cols].median()
    df_transformed[numeric_cols] = df_transformed[numeric_cols].fillna(medians)
    
    # Features for modeling
    features_for_modeling = [
        # Original features
        'Total_Amount', 'duration',
        'Amount_Funded_By_Lender',
        
        # High importance ratios
        'repayment_to_loan_ratio',
        'daily_repayment',
        'daily_principal',
        'daily_interest',
        
        # Customer metrics
        'loan_count',
        'unique_loans',
        'avg_loan_amount',
        'total_borrowed',
        'avg_duration',
        'max_duration',
        'duration_std',
        'repeat_ratio',
        
        # Risk indicators
        'interest_rate',
        'funding_ratio',
        'funding_gap_ratio'
    ]
    
    # Add economic features if available
    if econ_df is not None:
        economic_features = [
            'inflation_rate',
            'exchange_rate',
            'real_interest_rate',
            'deposit_rate',
            'lending_rate',
            'interest_spread',
            'unemployment_rate',
            'interest_to_market_ratio',
            'relative_funding_cost'
        ]
        features_for_modeling.extend(economic_features)
    
    return df_transformed[features_for_modeling].clip(-1e15, 1e15)

# Usage:
# First prepare economic data
econ_df_prepared = prepare_economic_data(econ_in_df)

# Then create features
X = create_aggregated_features(train_df, econ_df=econ_df_prepared)

print("Final feature set shape:", X.shape)
print("\nFeatures included:", X.columns.tolist())

In [ ]:
viz = Visualizer()
viz.plot_histograms(X, num_cols=3).show()

In [ ]:
# Train-test split
X_train, X_valid, y_train, y_valid = train_test_split(
    X, train_df['target'], 
    stratify=train_df['target'],
    test_size=0.2,
    random_state=42
)

# Check final feature set
print("Final feature set shape:", X.shape)
print("\nFeatures included:", X.columns.tolist())

In [ ]:
def create_focused_features(train_df, test_df=None):
    """
    Creates simplified, focused features based on feature importance analysis
    """
    def safe_divide(a, b):
        """Safe division handling zeros"""
        return np.divide(a, b, out=np.zeros_like(a), where=b!=0)
    
    # Create core features (common important features across models)
    def calculate_core_features(df):
        df = df.copy()
        
        # Most important features across all models
        df['repayment_to_loan_ratio'] = safe_divide(df['Total_Amount_to_Repay'], df['Total_Amount'])
        df['daily_interest'] = safe_divide(df['Total_Amount_to_Repay'] - df['Total_Amount'], df['duration'])
        df['interest_rate'] = safe_divide(df['Total_Amount_to_Repay'] - df['Total_Amount'], df['Total_Amount'])
        
        return df
    
    # Basic customer aggregations
    customer_agg = train_df.groupby('customer_id').agg({
        'tbl_loan_id': 'count',          # Number of loans
        'Total_Amount': ['mean', 'sum'],  # Borrowing behavior
    }).round(3)
    
    # Flatten column names
    customer_agg.columns = [
        'loan_count',
        'avg_loan_amount', 
        'total_borrowed'
    ]
    
    # Process data
    if test_df is not None:
        df_to_transform = test_df.copy()
    else:
        df_to_transform = train_df.copy()
    
    # Calculate features and merge
    df_transformed = calculate_core_features(df_to_transform)
    df_transformed = df_transformed.merge(customer_agg, on='customer_id', how='left')
    
    # Handle missing values
    df_transformed = df_transformed.fillna(0)
    
    # Final feature list
    features_for_modeling = [
        # Original features
        'Total_Amount',
        'duration',
        
        # Key ratios (most important across models)
        'repayment_to_loan_ratio',
        'daily_interest',
        'interest_rate',
        
        # Basic customer metrics
        'loan_count',
        'avg_loan_amount',
        'total_borrowed'
    ]
    
    return df_transformed[features_for_modeling]

In [ ]:
# Train-test split
X_train, X_valid, y_train, y_valid = train_test_split(
    X, train_df['target'], 
    stratify=train_df['target'],
    test_size=0.2,
    random_state=42
)

# Check final feature set
print("Final feature set shape:", X.shape)
print("\nFeatures included:", X.columns.tolist())

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import (
    f1_score, classification_report, confusion_matrix,
    roc_curve, precision_recall_curve, auc, roc_auc_score
)
import lightgbm as lgb
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
import matplotlib.pyplot as plt
import seaborn as sns

def plot_feature_importance(feature_importance, title):
    """
    Plot feature importance as percentages with enhanced visualization
    """
    # Calculate percentage importance
    feature_importance['importance_pct'] = feature_importance['importance'] / feature_importance['importance'].sum() * 100
    
    plt.figure(figsize=(12, 6))
    colors = sns.color_palette("viridis", n_colors=10)
    
    ax = sns.barplot(
        data=feature_importance.head(10),
        x='importance_pct',
        y='feature',
        palette=colors
    )
    
    plt.title(f'Top 10 Feature Importance - {title}', fontsize=12, pad=20)
    plt.xlabel('Importance (%)')
    plt.ylabel('Features')
    
    # Add percentage labels
    for i, v in enumerate(feature_importance['importance_pct'].head(10)):
        ax.text(v + 0.5, i, f'{v:.1f}%', va='center')
    
    plt.tight_layout()
    plt.show()

def plot_validation_metrics(y_valid, valid_preds, valid_probs=None, model_name=""):
    """
    Comprehensive validation metrics visualization
    """
    fig = plt.figure(figsize=(20, 5))
    
    # 1. Confusion Matrix
    plt.subplot(1, 3, 1)
    cm = confusion_matrix(y_valid, valid_preds)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=['No Default', 'Default'],
                yticklabels=['No Default', 'Default'])
    plt.title(f'Confusion Matrix - {model_name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    
    if valid_probs is not None:
        # 2. ROC Curve
        plt.subplot(1, 3, 2)
        fpr, tpr, _ = roc_curve(y_valid, valid_probs)
        roc_auc = roc_auc_score(y_valid, valid_probs)
        
        plt.plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], 'k--', label='Random')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'ROC Curve - {model_name}')
        plt.legend(loc="lower right")
        
        # 3. Precision-Recall Curve
        plt.subplot(1, 3, 3)
        precision, recall, _ = precision_recall_curve(y_valid, valid_probs)
        pr_auc = auc(recall, precision)
        
        plt.plot(recall, precision, label=f'PR curve (AUC = {pr_auc:.2f})')
        plt.xlabel('Recall')
        plt.ylabel('Precision')
        plt.title(f'Precision-Recall Curve - {model_name}')
        plt.legend(loc="lower left")
    
    plt.tight_layout()
    plt.show()

def train_evaluate_models(X_train, X_valid, y_train, y_valid):
    """
    Train and evaluate multiple models with enhanced visualization
    """
    # Calculate class weights
    n_negative = len(y_train[y_train == 0])
    n_positive = len(y_train[y_train == 1])
    scale_pos_weight = n_negative / n_positive
    
    # Initialize scaler
    scaler = RobustScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_valid_scaled = scaler.transform(X_valid)

    # Initialize models with optimal parameters for credit scoring
    models = {
       'RandomForest': RandomForestClassifier(
           n_estimators=200,
           max_depth=7,
           min_samples_split=10,
           min_samples_leaf=4, 
           max_features='sqrt',
           class_weight='balanced',
           random_state=42,
           n_jobs=-1
       ),
       
       'XGBoost': xgb.XGBClassifier(
           n_estimators=200,
           max_depth=6,
           learning_rate=0.1,
           subsample=0.8,
           colsample_bytree=0.8,
           min_child_weight=3,
           scale_pos_weight=scale_pos_weight,  # Calculated from data
           random_state=42,
           n_jobs=-1
       ),
       
       'LightGBM': lgb.LGBMClassifier(
           scale_pos_weight=scale_pos_weight,  # Calculated from data
           random_state=42,
           n_jobs=-1
       )
    }

    
    results = {}
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    for name, model in models.items():
        print(f"\n{'='*50}")
        print(f"Training {name}...")
        print('='*50)
        
        try:
            # Cross-validation
            cv_scores = cross_val_score(
                model, X_train_scaled, y_train,
                cv=cv, scoring='f1', n_jobs=-1
            )
            
            # Model training
            if name == 'XGBoost':
                model.fit(
                    X_train_scaled, y_train,
                    eval_set=[(X_valid_scaled, y_valid)],
                    early_stopping_rounds=50,
                    verbose=False
                )
            else:
                model.fit(X_train_scaled, y_train)
            
            # Predictions
            train_preds = model.predict(X_train_scaled)
            valid_preds = model.predict(X_valid_scaled)
            valid_probs = model.predict_proba(X_valid_scaled)[:, 1] if hasattr(model, 'predict_proba') else None
            
            # Store results
            results[name] = {
                'cv_f1_mean': cv_scores.mean(),
                'cv_f1_std': cv_scores.std(),
                'train_f1': f1_score(y_train, train_preds),
                'valid_f1': f1_score(y_valid, valid_preds),
                'model': model,
                'feature_importance': None
            }
            
            # Print metrics
            print("\nModel Performance:")
            print(f"CV F1 Score: {cv_scores.mean():.4f} (±{cv_scores.std()*2:.4f})")
            print(f"Train F1 Score: {results[name]['train_f1']:.4f}")
            print(f"Valid F1 Score: {results[name]['valid_f1']:.4f}")
            print("\nClassification Report:")
            print(classification_report(y_valid, valid_preds))
            
            # Plot validation metrics
            plot_validation_metrics(y_valid, valid_preds, valid_probs, name)
            
            # Feature importance
            if hasattr(model, 'feature_importances_') and name != 'BaselineRandom':
                feature_importance = pd.DataFrame({
                    'feature': X_train.columns,
                    'importance': model.feature_importances_
                }).sort_values('importance', ascending=False)
                results[name]['feature_importance'] = feature_importance
                plot_feature_importance(feature_importance, name)
            
        except Exception as e:
            print(f"Error training {name}: {str(e)}")
            continue
    
    return results, scaler

# Train and evaluate models
results, scaler = train_evaluate_models(X_train, X_valid, y_train, y_valid)

# Find best model
best_model_name = max(results, key=lambda k: results[k]['valid_f1'])
best_model = results[best_model_name]['model']

print("\nBest Model Summary:")
print("="*50)
print(f"Best Model: {best_model_name}")
print(f"Validation F1 Score: {results[best_model_name]['valid_f1']:.4f}")

In [5]:
import datetime

def create_submission(test_df, model, scaler, feature_columns):
   """
   Create submission file for the competition
   """
   econ_df_prepared = prepare_economic_data(econ_in_df)
    
   # Prepare test features using the same aggregation function used for training
   X_test = create_aggregated_features(test_df, econ_df=econ_df_prepared)
   
   # Scale features using the same scaler fitted on training data
   X_test_scaled = scaler.transform(X_test[feature_columns])
   
   # Make predictions
   predictions = model.predict(X_test_scaled)
   
   # Create submission dataframe with required format
   submission = pd.DataFrame({
       'ID': test_df['ID'],
       'Target': predictions.astype(int)  # Ensure predictions are integers (0 or 1)
   })
   
   # Save submission to CSV
   submission_filename = f'submission_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
   submission.to_csv(submission_filename, index=False)
   
   # Print summary
   print(f"\nSubmission Summary:")
   print("-" * 30)
   print(f"Total predictions: {len(predictions)}")
   print(f"Predicted defaults (1): {sum(predictions == 1)}")
   print(f"Predicted non-defaults (0): {sum(predictions == 0)}")
   print(f"\nSubmission saved as: {submission_filename}")
   
   return submission


# Create submission
submission = create_submission(test_df, best_model, scaler, X_train.columns)

# Verify submission format
print("\nSubmission Format Verification:")
print("-" * 30)
print("Submission shape:", submission.shape)
print("\nFirst few rows:")
print(submission.head())
print("\nValue counts:")
print(submission['Target'].value_counts())

# Additional verification checks
assert submission.shape[1] == 2, "Submission should have exactly 2 columns"
assert submission.columns.tolist() == ['ID', 'Target'], "Columns should be 'ID' and 'target'"
assert submission['Target'].isin([0, 1]).all(), "Predictions should be binary (0 or 1)"
assert not submission.isnull().any().any(), "Submission contains null values"

NameError: name 'test_df' is not defined